# Virtuous Interview Exam

> Monday August 14th, 2023

# Requirements

> Virtuous Data Schema, changes that require ETL have been highlighted <br>

<br>

![alternative text](images/virtuous_requirements.png)

## Summary
> I need to perform a ETL of 3 spreadsheets into SQL tables that match the Virtuous schema
- Contacts: A table of contacts, with 1 or 2 constituents per row
- Gifts: Donations from individual constituents
- Contact Methods: Contact information for individual constituents

In [1]:
#| default_exp utils

# Previewing The datasets
> Loading Datasets using Pandas

In [2]:
#| export
import pandas as pd
import re

In [3]:
#| export
contact_methods = pd.read_csv('data/contact_methods.csv')
contacts = pd.read_csv('data/contacts.csv')
gifts = pd.read_csv('data/gifts.csv')

Filling na values with empty strings for consistency

In [4]:
#| export
contact_methods.fillna('', inplace=True)
contacts.fillna('', inplace=True)
gifts.fillna('', inplace=True)

In [5]:
contacts.head()

,Number,Company Name,First Name,Last Name,Street,City,State,Postal,Phone,E-mail,Remarks,Deceased?
0,653377813-7,,Karita & Kelvin,Lumbers,4 Bunting Parkway,Washington,DC,20535-871,kklumbers@ yahoo.co,,Is anonymous,
1,390551098-7,,Helga,Benech,48684 Jenifer Way,Las Vegas,NV,89130,,ebenech1@goodreads.com,,
2,093004505-X,,Masha,,353 Schmedeman Park,Indianapolis,IN,,,577-374-96523,,
3,729707142-0,A Company Co.,,,2055 Lakewood Parkway,Camden,NJ,8104,,,,No
4,488464926-5,,Hoyt,Castille,37 8th Trail,Grand Rapids,MI,49560,,fcastille4@timesonline.co.uk,,No


In [6]:
gifts.head()

,donor_number,gift_id,first_name,last_name,amount received,date,fund_id,credit card type,payment method,pledge_number,notes
0,848348568-0,95196378.0,Mannie,Turpin,$4.15,3/4/2019,,,PayPal,,
1,729707142-0,95196889.0,Cymbre,Cross,2.3648,3/5/2019,ChildSponsorship,,check,,
2,687119652-8,95197689.0,Ruggiero,Makepeace,$1.31,3/7/2019,,,cash,,
3,653377813-7,95198998.0,Karita,Lumbers,$2.04,3/10/2019,,American Ex,credit card,,In honor of Mannie Turpin
4,390551098-7,95198999.0,Helga,Benech,$5.80,2019/1/10,,,cash,89752384.0,


In [7]:
contact_methods.head()

,donor_number,Phone,E-mail,Fax
0,653377813-7,832-442-4988,,
1,390551098-7,,ebenech1@goodreads.com,
2,093004505-X,818-323-9865,,818-156-7985
3,729707142-0,,,
4,488464926-5,,fcastille4@timesonline.co.uk,


Before I begin the performing the analysis in each method's respective notebook, I'm going to standardize the column names and types across the 3 DataFrames so I can pass them to each notebook and save myself some tedious typing**<br>
<br>
** Also doing this to show usefullness of juptyer notebook in production environments :)

## Housekeeping 1
> Format Column Names
<br>

- Nothing worse than malformed column names, amiright?

### to_camel_case
> Apply Camel Casing to a string

In [8]:
#| export
def to_camel_case(s):
    # Remove all non-alphanumeric characters and replace with a space
    s = re.sub(r'[^a-zA-Z0-9]', ' ', s)
    
    # Split by space and capitalize the first letter of each word
    words = s.split()
    return ''.join(word.capitalize() for word in words)

### transform_cnames
> Rename DF cnames in place

In [9]:
#| export
def transform_cnames(df, func=to_camel_case):
    df.columns = df.columns.map(func)
    return None

In [10]:
#| export
for df in [contact_methods, contacts, gifts]:
    transform_cnames(df)

In [11]:
#| hide
contacts.columns

Index(['Number', 'CompanyName', 'FirstName', 'LastName', 'Street', 'City',
       'State', 'Postal', 'Phone', 'EMail', 'Remarks', 'Deceased'],
      dtype='object')

## Housekeeping 2
> Cleaning the column types

I identified 2 columns on the gift table that should be ints, replacing those values

In [12]:
#| export
int_cols = ['GiftId', 'PledgeNumber']

In [13]:
#| export
gifts[int_cols] = gifts[int_cols].replace({'':0}).astype(int)

Looks like AmountRecieved should really be a float, I'm removing any special characters (besides dashes and periods) and converting to a float :)

In [14]:
#| export
gifts['AmountReceived'] = gifts.AmountReceived.apply(lambda x: float(re.sub(r'[^a-zA-Z0-9\.-]', '', x)))

Replacing 0 with unique id

In [15]:
gifts

,DonorNumber,GiftId,FirstName,LastName,AmountReceived,Date,FundId,CreditCardType,PaymentMethod,PledgeNumber,Notes
0,848348568-0,95196378,Mannie,Turpin,4.1500,3/4/2019,,,PayPal,0,
1,729707142-0,95196889,Cymbre,Cross,2.3648,3/5/2019,ChildSponsorship,,check,0,
2,687119652-8,95197689,Ruggiero,Makepeace,1.3100,3/7/2019,,,cash,0,
3,653377813-7,95198998,Karita,Lumbers,2.0400,3/10/2019,,American Ex,credit card,0,In honor of Mannie Turpin
4,390551098-7,95198999,Helga,Benech,5.8000,2019/1/10,,,cash,89752384,
5,848348568-0,95296677,Mannie,Turpin,9.2800,3/20/2019,"General, ReliefFund",,PayPal,0,
6,029456846-8,95298831,Romy,Doley,5.0000,3/24/2019,,,check,0,ACH check #7687
7,093004505-X,95298845,Masha,Butt Gow,4.8300,4/9/2019,,,check,0,
8,315297729-8,95298997,Idell,Mouncey,7.0000,4/12/2019,SchoolSupplies2019,,check,0,
9,809975531-Y,0,Adeline,Shakespeare,8.4800,8/14/2019,,AMEX,credit card,0,


In [16]:
#| export
gifts.loc[ gifts.PledgeNumber == 0, 'PledgeNumber'] = gifts[gifts.PledgeNumber == 0].index
gifts.loc[ gifts.GiftId == 0, 'GiftId'] = gifts[gifts.GiftId == 0].index

In [17]:
#| export
gifts = gifts.rename(columns={'PledgeNumber': 'LegacyPledgeID', 'GiftId': 'LegacyGiftID'})

## Housekeeping 3
> Identify and clean any pieces of data in the wrong column

In [18]:
#| hide
contacts[['Phone', 'EMail']].head(3)

,Phone,EMail
0,kklumbers@ yahoo.co,
1,,ebenech1@goodreads.com
2,,577-374-96523


Creating a function that uses regular expressions to identify a string as a phone or a email

In [19]:
#| export
def classify_phone_email(value):
    if "@" in value:
        return "email"
    if re.search(r'\d{3}-\d{3}-\d{4}', value):
        return "phone"
    return None

Looping over phone & email records.<br>
<br>
If a phone and/or email are identified in the wrong column, they will be swapped

In [20]:
#| export
for index, row in contacts.iterrows():
    phone_classification = classify_phone_email(row['Phone'])
    email_classification = classify_phone_email(row['EMail'])

    if phone_classification == "email":
        contacts.at[index, 'EMail'] = row['Phone']
        contacts.at[index, 'Phone'] = ''

    if email_classification == "phone":
        contacts.at[index, 'Phone'] = row['EMail']
        contacts.at[index, 'EMail'] = ''

In [21]:
#| hide
contacts[['Phone', 'EMail']].head(3)

,Phone,EMail
0,,kklumbers@ yahoo.co
1,,ebenech1@goodreads.com
2,577-374-96523,


## Housekeeping 4
> Consolidating contacts table <br>
<br>
- Searching for missing users <br>
- Splitting / joining households


### Checking for Missing records <br><br>

Checking for any DonorNumbers that are NOT IN contacts Number...

In [22]:
#| hide
(~gifts.DonorNumber.isin(contacts.Number.unique())).any()

True

Found 1! <br>
<br>
Doing the same for contact_methods

In [23]:
#| hide
(~contact_methods.DonorNumber.isin(contacts.Number.unique())).any()

False

Didn't find any there <br>
Going to extract the missing records from the gifts table

In [24]:
#| export
donors_not_in_contacts = gifts.loc[~gifts.DonorNumber.isin(contacts.Number.unique()), :]

In [25]:
#| hide
donors_not_in_contacts

,DonorNumber,LegacyGiftID,FirstName,LastName,AmountReceived,Date,FundId,CreditCardType,PaymentMethod,LegacyPledgeID,Notes
9,809975531-Y,9,Adeline,Shakespeare,8.48,8/14/2019,,AMEX,credit card,9,
27,809975531-Y,27,Adeline,Shakespeare,7.58,8/14/2019,"Color run, ChildSponsorship",Mastercard,credit card,27,


Adding to Shakespeare to our contacts!

In [26]:
#| export
contacts = pd.concat([
    contacts,
    # Dataframe of donors not in contacts
    pd.DataFrame(donors_not_in_contacts[['DonorNumber', 'FirstName', 'LastName']]
                 .drop_duplicates()
                 .rename(columns={'DonorNumber': 'Number'})
                 .drop_duplicates()
                 .to_dict('records'))
])

## Spliting rows with Multiple People

In [27]:
#| hide
contacts.FirstName.head(1)

0    Karita & Kelvin
Name: FirstName, dtype: object

Split the names on ' & ' or ' and ', then expand the resulting lists into new rows

In [28]:
#| export
contacts[['FirstName', 'SecondaryFirstName']] = contacts['FirstName'].str.split(' & | and ', expand=True).fillna('')

Aditionally, going to look for any records where there's a duplicated `Number` <br>
<br>
I'm assuming `Number` is a primary key for a household or organization

In [29]:
#| export
records_to_join = contacts.loc[contacts.Number.duplicated(), :].to_dict(orient='records')

 Removing duplicates from contacts before joining

In [30]:
#| export
contacts = contacts.loc[~contacts.Number.duplicated(), :]

Adding dupliates as secondary contacts

In [31]:
#| export
for record in records_to_join:
    contacts.loc[contacts.Number.isin([record['Number']]), ['SecondaryFirstName', 'SecondaryLastName']] = [record['FirstName'], record['LastName']]

Adding Secondary Last Name for the appropriate users

In [32]:
#| export
contacts['SecondaryLastName'] = contacts.apply(lambda x: x['LastName'] if x['SecondaryLastName'] == '' and x['SecondaryFirstName'] != '' else x['SecondaryLastName'], axis=1)

Initializing new columns

In [33]:
#| export
contacts[['LegacyIndividualId', 'SecondaryLegacyIndividualId']] = None

In [34]:
#| export
contacts.reset_index(inplace=True, drop=True)

Adding id since none was provided

In [35]:
#| export
id = 0
for index, row in contacts.iterrows():
    contacts.loc[index, 'LegacyIndividualId'] = id
    id += 1
    if row['SecondaryFirstName'] != '':
        contacts.loc[index, 'SecondaryLegacyIndividualId'] = id
        id += 1


In [36]:
#| export
contacts.fillna('', inplace=True)

## Housekeeping 5
> Cleaning Records with blank first or last names

Checking for records where FirstName  and/or LastName is blank


In [37]:
#| export
blank_name_records = ((contacts.FirstName == '') | (contacts.LastName == ''))

Previewing the blank name records

In [38]:
contacts.loc[((contacts.FirstName == '') | (contacts.LastName == '')), :]

,Number,CompanyName,FirstName,LastName,Street,City,State,Postal,Phone,EMail,Remarks,Deceased,SecondaryFirstName,SecondaryLastName,LegacyIndividualId,SecondaryLegacyIndividualId
2,093004505-X,,Masha,,353 Schmedeman Park,Indianapolis,IN,,577-374-96523,,,,,,3,
3,729707142-0,A Company Co.,,,2055 Lakewood Parkway,Camden,NJ,8104,,,,No,,,4,
7,029456846-8,,,,608 Old Shore Alley,Marietta,GA,30066,,jdoley6@telegraph.co.uk,,,,,9,


Before I delete the records I'm going to check if the names are present on the gift table <br>
<br>
I'm going to start by getting the unique Numbers that the records belong too

In [39]:
#| export
blank_name_numbers= contacts.loc[blank_name_records, 'Number']

In [40]:
gifts.loc[gifts.DonorNumber.isin(blank_name_numbers), ['DonorNumber', 'FirstName', 'LastName']].drop_duplicates().head(5)

,DonorNumber,FirstName,LastName
1,729707142-0,Cymbre,Cross
6,029456846-8,Romy,Doley
7,093004505-X,Masha,Butt Gow
19,729707142-0,Cymbre,


The names are present on the gifts table!

In [41]:
#| export
gift_name_records = gifts.loc[gifts.DonorNumber.isin(blank_name_numbers), ['DonorNumber', 'FirstName', 'LastName']].drop_duplicates()

Removing the invalid record

In [42]:
#| export
gift_name_records = gift_name_records.loc[((gift_name_records.FirstName != '') & (gift_name_records.LastName != '')), :]
gift_name_records

,DonorNumber,FirstName,LastName
1,729707142-0,Cymbre,Cross
6,029456846-8,Romy,Doley
7,093004505-X,Masha,Butt Gow


Updating the records that previousuly had a blank first or last name

In [43]:
#| export
for _, row in gift_name_records.iterrows():
    contacts.loc[contacts['Number'] == row['DonorNumber'], ['FirstName', 'LastName']] = [row['FirstName'], row['LastName']]

All the records valid names now!

In [44]:
contacts.loc[blank_name_records, :]

,Number,CompanyName,FirstName,LastName,Street,City,State,Postal,Phone,EMail,Remarks,Deceased,SecondaryFirstName,SecondaryLastName,LegacyIndividualId,SecondaryLegacyIndividualId
2,093004505-X,,Masha,Butt Gow,353 Schmedeman Park,Indianapolis,IN,,577-374-96523,,,,,,3,
3,729707142-0,A Company Co.,Cymbre,Cross,2055 Lakewood Parkway,Camden,NJ,8104,,,,No,,,4,
7,029456846-8,,Romy,Doley,608 Old Shore Alley,Marietta,GA,30066,,jdoley6@telegraph.co.uk,,,,,9,


## Housekeeping 6
> Adding Contact Name

In [45]:
contacts.head(3)

,Number,CompanyName,FirstName,LastName,Street,City,State,Postal,Phone,EMail,Remarks,Deceased,SecondaryFirstName,SecondaryLastName,LegacyIndividualId,SecondaryLegacyIndividualId
0,653377813-7,,Karita,Lumbers,4 Bunting Parkway,Washington,DC,20535-871,,kklumbers@ yahoo.co,Is anonymous,,Kelvin,,0,1
1,390551098-7,,Helga,Benech,48684 Jenifer Way,Las Vegas,NV,89130,,ebenech1@goodreads.com,,,,,2,
2,093004505-X,,Masha,Butt Gow,353 Schmedeman Park,Indianapolis,IN,,577-374-96523,,,,,,3,


Creating a function to update the contact name depending on the individuals on the household record

In [46]:
#| export
def set_contact_name(row):
    if row['LastName'] == row['SecondaryLastName']:
        return row['FirstName'] + ' & ' + row['SecondaryFirstName'] + ' ' + row['LastName']
    elif row['SecondaryFirstName'] != '':
        return row['FirstName'] +  ' ' + row['LastName'] + ' & ' + row['SecondaryFirstName'] + ' ' + row['SecondaryLastName']
    else:
        return row['FirstName'] + ' ' + row['LastName']

In [47]:
#| export
contacts['ContactName'] = contacts.apply(set_contact_name, axis=1)

## Housekeeping 7
> Splitting Project Codes

In [48]:
#| export
project_codes = gifts.FundId.str.split(', ', expand=True)

In [49]:
#| export
project_codes.head(3)

,0,1
0,,None
1,ChildSponsorship,None
2,,None


In [50]:
#| export
gifts[['Project1Code', 'Project2Code']] = project_codes

In [51]:
#| export
gifts = gifts.loc[:, gifts.columns.drop('FundId')].copy()

## Housekeeping 8
> Formating date

In [52]:
df['Date'].head(7)

0     3/4/2019
1     3/5/2019
2     3/7/2019
3    3/10/2019
4    2019/1/10
5    3/20/2019
6    3/24/2019
Name: Date, dtype: object

Since Dates are in different formats I'm going to create a custom parser

In [53]:
#| export
from datetime import datetime
def custom_parser(date_str):
    try:
        return datetime.strptime(date_str, '%m/%d/%Y')
    except ValueError:
        return datetime.strptime(date_str, '%Y/%m/%d')

Applying parser

In [54]:
gifts['GiftDate'] = gifts['Date'].apply(custom_parser)

Dropping old date columns

In [ ]:
gifts = gifts.loc[:, gifts.columns.drop('Date')].copy()

# Export

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export('00_Setup.ipynb')

In [ ]:
#| hide
nbdev.nbdev_export('00_Setup.ipynb')

<br>